# 1.1 Movies Database

Take the movies dataset and turn it into a single `sqlite` database. It should have one table for each csv file in the movies dataset

In [61]:
import sqlite3
import pandas as pd

files = ['names', 'movies', 'ratings', 'title_principals']

movies_set = sqlite3.connect('data/movies_set.sqlite')

def sqlize_csv(file):
    df = pd.read_csv('data/'+file+'.csv')
    df.to_sql(
        name=file,
        con=movies_set, 
        schema=None, 
        if_exists='replace', 
        index=True, 
    )
    
for file in files:
    print('Saving', file)
    sqlize_csv(file)

Saving names
Saving movies


/home/jlmy/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Saving ratings
Saving title_principals


In [62]:
# c = movies_set.cursor()
# c.execute("SELECT name FROM sqlite_master WHERE type='table';")
# c.fetchall()

# 1.2 Queries

**1.2.1** Use a single query to pull the original title of movies with a budget above $5m

**1.2.2** Use a query to pull the english-language films with the word `war` in their title

**1.2.3** Left join the average ratings from the `ratings` table onto the `movies_metadata` table, so you can have a relation between budget and rating. Hint: use a subquery.

In [63]:
movies = pd.read_csv('data/movies.csv')

def get_currency(x):
    if isinstance(x, str):
        return(x.split(' ')[0])
    return('')

movies['Currency'] = movies['budget'].apply(get_currency)

currencies = movies['Currency'].unique().tolist()

def clean_currency(x):
    if isinstance(x, str):
        xt = x
        for currency in currencies:
            xt = xt.replace(currency, '')
        return(xt.replace(',', ''))
    return(x)

/home/jlmy/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
movies['budget'] = movies['budget'].apply(clean_currency).astype(float)
movies['budget'] = movies['budget'].fillna(0.)
movies.to_sql(
    name='movies',
    con=movies_set, 
    schema=None, 
    if_exists='replace', 
    index=True, 
)

**1.2.1**

In [65]:
def query_movies(q):
    return pd.read_sql_query(q, con=movies_set)

In [66]:
five_mil = query_movies("SELECT title FROM movies WHERE budget > 5000000")
five_mil

,title
0,Metropolis
1,Napoleone
2,La regola del gioco
3,Kate & Leopold
4,La cittadella degli eroi
...,...
9641,Aakashaganga II
9642,Munthiri Monchan
9643,Upin & Ipin: Keris Siamang Tunggal
9644,Kaithi


**1.2.2**

In [67]:
q = """
SELECT title 
FROM movies 
    WHERE language = 'English' 
    AND title LIKE '% war %'
"""

eng_war = query_movies(q)
eng_war

,title
0,The War Against Mrs. Hadley
1,Linea di fuoco - War zone
2,Afganistan - The last war bus (L'ultimo bus di...
3,The War Bride
4,The War Within
5,Der Fluss war einst ein Mensch
6,The War of 1812
7,Cinematic Titanic: War of the Insects
8,The Civil War on Drugs
9,The War I Knew


**1.2.3** Left join the average ratings from the ratings table onto the movies_metadata table, so you can have a relation between budget and rating. Hint: use a subquery.



# 2. Baseball Database

The [Baseball Database](http://www.seanlahman.com/baseball-archive/statistics/) has an sqlite version. Download it for these exercises.

**2.1** Which player has had the most homeruns?

**2.2** Is there a relation between how many homeruns a player has made in a year and his salary that year? Pull both colums together in a single query

